# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
#funcion que nos ayuda a limpiar las cuidades
def limpiarciudades(x):
        equipos2palabras=["Golden Knights","Blue Jackets","Maple Leafs","Red Wings"]
        equipo_de_2_palabras=False
        #verifica si el nombre esta en alguna de las ciudades de 2 palabras
        for equipo in equipos2palabras:
            if equipo in x:
                equipo_de_2_palabras=True
        if equipo_de_2_palabras:
            #si esta,tomamos las ultimas 2 palabras unicamente (con un split y luego un join para volverlas a pegar)
            return " ".join(x.split(" ")[-2:])
        else:
            #retornamos solo el equipo
            return x.split(" ")[-1]


def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    #empezamos limpiando la poblacion
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #retiramos las notas (demarcadas con []) usando regex
    cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    #remplazamos con el fin de separar los equipos de una misma cuidad
    cities["NHL"] = cities["NHL"].replace({"Rangers Islanders Devils": "Rangers,Islanders,Devils",
                                           "Kings Ducks": "Kings,Ducks" })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    #los separamos
    cities = cities.explode("NHL")
    #seleccionamos unicamente la informacion que nos interesa:
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]','NHL']]
    #limpiamos los datos vacios quitando las filas innecesarias:
    cities = cities[(cities["NHL"] != "—") & (cities["NHL"] != "")]

    #limpiamos los datos de los equipos
    #nos piden solo los datos de 2018,ademas, hay algunos equipos que no estan en cities pero si en nhl, 
    # asi que los quitamos dando entonces:
    nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0)
    
    #quitamos alguna basura que hay en los nombres de los equipos y algunos espacios que pueda haber:
    nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
    #limpiamos las cuidades del principiio del nombre usando la funcion limpiarciudades 
    nhl_df["team"] = nhl_df['team'].apply(lambda x:  limpiarciudades(x))
    #seleccionamos lo que nos interesa
    nhl_df=nhl_df [["team", "W", "L"]]
    #mergeamos teniendo el cuenta los nomrbes del equipo 
    final = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
    #sacamos el Win/Loss ratio haciendo el respectivo calculo y lo añadmos a una columna
    #importante: los tipos de datos deben estar bien dedfinidos para no tener porblemas  
    final["W-L%"] = final["W"].astype("int")/(final["W"].astype("int") + final["L"].astype("int"))
    final["Population (2016 est.)[8]"] = final["Population (2016 est.)[8]"].astype("float")
    final["W-L%"] = final["W-L%"].astype("float")
    #eliimiinamos columnas repetidas
    final.loc[final["Metropolitan area"] == "New York City", "W-L%"] = 0.5182013333333334 # mean of NY W-L%
    final.loc[final["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.6228945 # mean of LA W-L%
    final = final.drop_duplicates(subset="Metropolitan area").reset_index()
    final = final.drop(columns="index")
    population_by_region = final["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = final["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    final.to_csv("final.csv")
    resultado = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return resultado[0]

nhl_correlation()


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
#funcion que nos ayuda a limpiar las cuidades
def limpiarciudades1(x):
        equipos2palabras=["Trail Blazers"]
        equipo_de_2_palabras=False
        #verifica si el nombre esta en alguna de las ciudades de 2 palabras
        for equipo in equipos2palabras:
            if equipo in x:
                equipo_de_2_palabras=True
        if equipo_de_2_palabras:
            #si esta,tomamos las ultimas 2 palabras unicamente (con un split y luego un join para volverlas a pegar)
            return " ".join(x.split(" ")[-2:])
        else:
            #retornamos solo el equipo
            return x.split(" ")[-1]


def nba_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    #empezamos limpiando la poblacion
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #retiramos las notas (demarcadas con []) usando regex
    cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    #remplazamos con el fin de separar los equipos de una misma cuidad
    
    cities["NBA"] = cities["NBA"].replace({"Knicks Nets": "Knicks,Nets",
                                           "Lakers Clippers": "Lakers,Clippers"})
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
    #los separamos
    cities = cities.explode("NBA")
    #seleccionamos unicamente la informacion que nos interesa:
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]','NBA']]
    #limpiamos los datos vacios quitando las filas innecesarias:
    cities = cities[(cities["NBA"] != "—") & (cities["NBA"] != "")]

    #limpiamos los datos de los equipos
    #nos piden solo los datos de 2018,ademas, hay algunos equipos que no estan en cities pero si en nhl, 
    # asi que los quitamos dando entonces:
    nba_df = nba_df[nba_df['year'] == 2018]

    #quitamos alguna basura que hay en los nombres de los equipos y algunos espacios que pueda haber:
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    #limpiamos las cuidades del principiio del nombre usando la funcion limpiarciudades 
    nba_df["team"] = nba_df['team'].apply(lambda x:  limpiarciudades1(x))
    #seleccionamos lo que nos interesa
    nba_df=nba_df [["team", "W", "L"]]
    #mergeamos teniendo el cuenta los nomrbes del equipo 
    final = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
    
    #sacamos el Win/Loss ratio haciendo el respectivo calculo y lo añadmos a una columna
    #importante: los tipos de datos deben estar bien dedfinidos para no tener porblemas  
    final["W-L%"] = final["W"].astype("int")/(final["W"].astype("int") + final["L"].astype("int"))
    final["Population (2016 est.)[8]"] = final["Population (2016 est.)[8]"].astype("float")
    final["W-L%"] = final["W-L%"].astype("float")
    #eliimiinamos columnas repetidas manualmente
    final.loc[final["Metropolitan area"] == "New York City", "W-L%"] = 0.34750000000000003 
    final.loc[final["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.46950000000000003 
    final = final.drop_duplicates(subset="Metropolitan area").reset_index()
    final = final.drop(columns="index")
    
    population_by_region = final["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = final["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    resultado = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return resultado[0]


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
#funcion que nos ayuda a limpiar las cuidades
def limpiarciudades2(x):
        equipos2palabras=["Red Sox","Blue Jays"]
        equipo_de_2_palabras=False
        #verifica si el nombre esta en alguna de las ciudades de 2 palabras
        for equipo in equipos2palabras:
            if equipo in x:
                equipo_de_2_palabras=True
        if equipo_de_2_palabras:
            #si esta,tomamos las ultimas 2 palabras unicamente (con un split y luego un join para volverlas a pegar)
            return " ".join(x.split(" ")[-2:])
        else:
            #retornamos solo el equipo
            return x.split(" ")[-1]


def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    #empezamos limpiando la poblacion
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #retiramos las notas (demarcadas con []) usando regex
    cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    #remplazamos con el fin de separar los equipos de una misma cuidad
    cities["MLB"] = cities["MLB"].replace({ 
                                           "Cubs White Sox": "Cubs,White Sox", 
                                           "Dodgers Angels": "Dodgers,Angels", 
                                           "Giants Athletics": "Giants,Athletics", 
                                           "Yankees Mets": "Yankees,Mets"})   
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
    #los separamos
    cities = cities.explode("MLB")
    #seleccionamos unicamente la informacion que nos interesa:
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]','MLB']]
    #limpiamos los datos vacios quitando las filas innecesarias:
    cities = cities[(cities["MLB"] != "—") & (cities["MLB"] != "")]

    #limpiamos los datos de los equipos
    #nos piden solo los datos de 2018,ademas, hay algunos equipos que no estan en cities pero si en nhl, 
    # asi que los quitamos dando entonces:
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    
    #limpiamos las cuidades del principiio del nombre usando la funcion limpiarciudades 
    mlb_df["team"] = mlb_df['team'].apply(lambda x:  limpiarciudades2(x))
    #seleccionamos lo que nos interesa
    mlb_df=mlb_df [["team", "W", "L"]]
    #mergeamos teniendo el cuenta los nomrbes del equipo 
    final = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
    #sacamos el Win/Loss ratio haciendo el respectivo calculo y lo añadmos a una columna
    #importante: los tipos de datos deben estar bien dedfinidos para no tener porblemas  
    final["W-L%"] = final["W"].astype("int")/(final["W"].astype("int") + final["L"].astype("int"))
    final["Population (2016 est.)[8]"] = final["Population (2016 est.)[8]"].astype("float")
    final["W-L%"] = final["W-L%"].astype("float")
    #eliimiinamos columnas repetidas
    final.loc[final["Metropolitan area"] == "New York City", "W-L%"] = 0.546 
    final.loc[final["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.5289999999999999 
    final.loc[final["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.525 
    final.loc[final["Metropolitan area"] == "Chicago", "W-L%"] = 0.482769 
    final.loc[final["Metropolitan area"] == "Boston", "W-L%"] = 0.666667 
    final = final.drop_duplicates(subset="Metropolitan area").reset_index()
    final = final.drop(columns="index")
    population_by_region = final["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = final["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    resultado = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return resultado[0]


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
#funcion que nos ayuda a limpiar las cuidades
def limpiarciudades2(x):
        equipos2palabras=["Red Sox","Blue Jays"]
        equipo_de_2_palabras=False
        #verifica si el nombre esta en alguna de las ciudades de 2 palabras
        for equipo in equipos2palabras:
            if equipo in x:
                equipo_de_2_palabras=True
        if equipo_de_2_palabras:
            #si esta,tomamos las ultimas 2 palabras unicamente (con un split y luego un join para volverlas a pegar)
            return " ".join(x.split(" ")[-2:])
        else:
            #retornamos solo el equipo
            return x.split(" ")[-1]


def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    #empezamos limpiando la poblacion
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #retiramos las notas (demarcadas con []) usando regex
    cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    #remplazamos con el fin de separar los equipos de una misma cuidad
    cities["NFL"] = cities["NFL"].replace({"Giants Jets": "Giants,Jets",
                                           "Rams Chargers": "Rams,Chargers",
                                           "49ers Raiders": "49ers,Raiders"
                                           })
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    #los separamos
    cities = cities.explode("NFL")
    #seleccionamos unicamente la informacion que nos interesa:
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]','NFL']]
    #limpiamos los datos vacios quitando las filas innecesarias:
    cities = cities[(cities["NFL"] != "—") & (cities["NFL"] != "")]

    #limpiamos los datos de los equipos
    #nos piden solo los datos de 2018,ademas, hay algunos equipos que no estan en cities pero si en nhl, 
    # asi que los quitamos dando entonces:
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    #limpiamos basura varia
    nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "", x))
    #limpiamos las cuidades del principiio del nombre usando la funcion limpiarciudades 
    nfl_df["team"] = nfl_df['team'].apply(lambda x:  limpiarciudades2(x))
    #seleccionamos lo que nos interesa
    nfl_df=nfl_df [["team", "W", "L"]]

    #mergeamos teniendo el cuenta los nomrbes del equipo 
    final = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
    #sacamos el Win/Loss ratio haciendo el respectivo calculo y lo añadmos a una columna
    #importante: los tipos de datos deben estar bien dedfinidos para no tener porblemas  
    final["W-L%"] = final["W"].astype("int")/(final["W"].astype("int") + final["L"].astype("int"))
    final["Population (2016 est.)[8]"] = final["Population (2016 est.)[8]"].astype("float")
    final["W-L%"] = final["W-L%"].astype("float")
    #eliimiinamos columnas repetidas
    final.loc[final["Metropolitan area"] == "New York City", "W-L%"] = 0.2815 # mean of NY W-L%
    final.loc[final["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.7815 # mean of LA W-L%
    final.loc[final["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.25 # mean of SF W-L%
    final = final.drop_duplicates(subset="Metropolitan area").reset_index()
    final = final.drop(columns="index")
    population_by_region = final["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = final["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    
    resultado = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return resultado[0]

nhl_correlation()